In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf
import QuantLib as ql
from numba import jit
from threading import Thread
import date_util
import sql_util

# Global Variables / Constants
db = 'PORTFOLIO_SNAPS'
# Start Date: 12/31/2019
db_start_date = ql.Date(31, 12, 2019)
db_beta_start_date = ql.Date(1, 11, 2016)

stock_list_filepath = r'/Users/jenniferzou/Downloads/Stocks Tool/Stocks.xlsx'
tickers_input_sheet = 'Portfolio'

# Dates
Date_Obj = date_util.myDates()

pos_date = Date_Obj.get_last_business_day(ql.Date.todaysDate())
hist_50d_date = Date_Obj.calendar.advance(pos_date, -50, ql.Days)
hist_200d_date = Date_Obj.calendar.advance(pos_date, -200, ql.Days)
hist_1m_date = Date_Obj.calendar.advance(pos_date, -1, ql.Months)
hist_10y_date = Date_Obj.calendar.advance(pos_date, -10, ql.Years)
hist_20y_date = Date_Obj.calendar.advance(pos_date, -20, ql.Years)
hist_30y_date = Date_Obj.calendar.advance(pos_date, -30, ql.Years)
print(f'POS_DATE: {pos_date.ISO()}')

DB_Obj = sql_util.myDB(db)

all_sec_info = []
all_tech_dfs = []
all_fund_dfs = []

POS_DATE: 2025-06-25


BackFill Stock Data Work

In [2]:

def load_historical_fundamentals_stock_profile(ticker, stock, start_dt, end_dt):
   
    query = f"""select * from SEC_INFO where ticker='{ticker}'"""
    sec_info = DB_Obj.read_sql_qry(query)
    sec_info.set_index('TICKER', inplace=True)
    query = f"""select POS_DATE, MARKET_CAP from TECHNICAL_SNAPS where ticker='{ticker}'"""
    technical_snaps = DB_Obj.read_sql_qry(query)

    df = stock.history(start=start_dt.ISO(), end=end_dt.ISO())
    df['POS_DATE'] = df.index.map(lambda x: Date_Obj.timestamp_to_ql(x).ISO())
    df['TICKER'] = ticker
    stock.financials.T.to_clipboard()
    if sec_info.loc[ticker, 'SEC_TYPE'] != 'ETF':
        financials = stock.financials.T[['Operating Income', 'Total Revenue',
                                         'EBIT', 'Net Interest Income', 'Basic EPS', 'Net Income', 'Interest Expense']]
        balance_sheet = stock.balance_sheet.T[['Total Debt','Cash And Cash Equivalents', 'Stockholders Equity', 'Current Assets', 'Inventory', 'Current Liabilities']]
        
        financials['FYE_DATE'] = financials.index.map(lambda x: Date_Obj.timestamp_to_ql(x).ISO())
        df = df.merge(financials, left_on='POS_DATE', right_on='FYE_DATE', how='left')
        balance_sheet['FYE_DATE_BS'] = balance_sheet.index.map(lambda x: Date_Obj.timestamp_to_ql(x).ISO())
        df = df.merge(balance_sheet, left_on='POS_DATE', right_on='FYE_DATE_BS', how='left')
        df = df.merge(technical_snaps, on='POS_DATE', how='left')

        df['FYE_DATE'] = df['FYE_DATE'].ffill()
        df['EBIT'] = df['EBIT'].ffill()
        df['NII'] = df['Net Interest Income'].ffill()

        df['Total Debt'] = df['Total Debt'].ffill()
        df['Cash And Cash Equivalents'] = df['Cash And Cash Equivalents'].ffill()
        df['Net Income'] = df['Net Income'].ffill()
        df['Stockholders Equity'] = df['Stockholders Equity'].ffill()
        df['Interest Expense'] = df['Interest Expense'].ffill()
        df['Operating Income'] = df['Operating Income'].ffill()
        df['Total Revenue'] = df['Total Revenue'].ffill()
        df['Current Assets'] = df['Current Assets'].ffill()
        df['Inventory'] = df['Inventory'].ffill()
        df['Current Liabilities'] = df['Current Liabilities'].ffill()

        df['EV'] = df['MARKET_CAP'] + df['Total Debt'] - df['Cash And Cash Equivalents']
    
        df['EPS'] = df['Basic EPS']
        df['Annual EPS Growth Rate'] = df['EPS'].pct_change()*100
        eps = stock.get_earnings_history()[['epsActual']]
        eps['POS_DATE'] = eps.index.map(lambda x: Date_Obj.timestamp_to_ql(x).ISO())
        df = df.merge(eps, on='POS_DATE', how='left')
        df['EPS'] = np.vectorize(lambda fye, qtr: qtr if qtr != None else fye) (df['epsActual'], df['EPS'])
        df['EPS'] = df['EPS'].ffill()
        df['Annual EPS Growth Rate'] = df['Annual EPS Growth Rate'].ffill()

        df['PE_RATIO'] = df['Close']/df['EPS']
        df['PEG_RATIO'] = np.vectorize(lambda pe, eps_growth: pe/eps_growth if eps_growth != 0 else None) \
            (df['PE_RATIO'],df['Annual EPS Growth Rate'])
        
        df['ROE'] = df['Net Income']/df['Stockholders Equity']
        df['DC_RATIO'] = df['Total Debt'] / (df['Total Debt']+df['Stockholders Equity'])    
        df['ICR'] = ((df['Net Income'] if 'Financial' in sec_info.loc[ticker, 'SECTOR'] else df['EBIT']) 
                     / df['Interest Expense']) if sec_info.loc[ticker, 'SEC_TYPE'] == 'EQUITY' else None
        df['EV_to_EBIT'] = (df['EV'] / (df['Net Income'] if 'Financial' in sec_info.loc[ticker, 'SECTOR'] else df['EBIT'])) \
                            if sec_info.loc[ticker, 'SEC_TYPE'] == 'EQUITY' else None
        df['OPERATING_MARGIN'] = df['Operating Income']/df['Total Revenue']
        df['QUICK_RATIO'] = (df['Current Assets']-df['Inventory']) / df['Current Liabilities']
        df['FWD_EPS'] = None
        df['FWD_PE'] = None
        df['ANALYST_RECS'] = None
        df['UPLOAD_STAT'] = 'BC'

    else:
        for col in ['FYE_DATE', 'EV', 'EBIT', 'NII', 'EPS','PE_RATIO', 'PEG_RATIO', 
                    'ROE', 'DC_RATIO', 'ICR', 'EV_to_EBIT','OPERATING_MARGIN', 'QUICK_RATIO']:
            df[f'{col}'] = None

    df['FWD_EPS'] = None
    df['FWD_PE'] = None
    df['ANALYST_RECS'] = None
    df['UPLOAD_STAT'] = 'BC'

    df = df[['POS_DATE', 'TICKER', 'FYE_DATE', 'EV', 'EBIT', 'NII', 'EPS', 
               'PE_RATIO', 'PEG_RATIO', 'ROE', 'DC_RATIO', 'ICR', 'EV_to_EBIT',
                'OPERATING_MARGIN', 'QUICK_RATIO', 'FWD_EPS', 'FWD_PE', 
                'ANALYST_RECS', 'UPLOAD_STAT' ]]
    all_fund_dfs.append(df)
    return 


In [3]:
def load_historical_technicals_stock_profile(ticker, stock, start_dt, end_dt, beta_df):
    
    start_dt_str = Date_Obj.calendar.advance(start_dt, -200, ql.Days).ISO()
    end_dt_str = end_dt.ISO()
    df = stock.history(start=start_dt_str, end=end_dt_str)
    df['POS_DATE'] = df.index.map(lambda x: Date_Obj.timestamp_to_ql(x))
    df['TICKER'] = ticker

    shares_df = stock.get_shares_full().to_frame(name='SHARES').sort_index()
    shares_df['POS_DATE'] = shares_df.index.map(lambda x: Date_Obj.dt_to_ql(x))
    df = df.merge(shares_df, on='POS_DATE', how='left')
    df['SHARES'] = df['SHARES'].ffill()
    df['MARKET_CAP'] = df['SHARES']*df['Close']

    df['SMA_50D'] = df["Close"].rolling(window=50).mean()
    df['SMA_200D'] = df["Close"].rolling(window=200).mean()
    df['EMA_12D'] = df["Close"].ewm(span=12, adjust=False).mean()
    df['EMA_26D'] = df["Close"].ewm(span=26, adjust=False).mean()
    df['EMA_50D'] = df["Close"].ewm(span=50, adjust=False).mean()
    df['EMA_200D'] = df["Close"].ewm(span=200, adjust=False).mean()
    df['VMA_20D'] = df['Volume'].rolling(window=20).mean()
    df['VMA_50D'] = df['Volume'].rolling(window=50).mean()
    df['VMA_200D'] = df['Volume'].rolling(window=200).mean()

    df['FOM'] = df['POS_DATE'].apply(lambda x: ql.Date(1, x.month(), x.year()))
    df = df.merge(beta_df[['FOM', f'{ticker}_BETA']], on='FOM', how='left')
    df['BETA'] = df[f'{ticker}_BETA']
    
    df['VWMA_50D'] = (df['Close']*df['Volume'].rolling(window=50).sum() / df['Volume'].rolling(window=50).sum())
    df['VWMA_200D'] = (df['Close']*df['Volume'].rolling(window=200).sum() / df['Volume'].rolling(window=200).sum())
    df['MACD_12D_26D'] = df['EMA_12D']-df['EMA_26D']
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    rs = gain.rolling(window=14).mean() / loss.rolling(window=14).mean()
    df['RSI_14D'] =  100 - (100 / (1 + rs))
    df['UPLOAD_STAT'] = 'BC'
    df['POS_DATE'] = df['POS_DATE'].apply(lambda pos_date: pos_date.ISO())

    df = df[['POS_DATE', 'TICKER', 'MARKET_CAP', 'Open', 'High', 'Low', 'Close', 
                'SMA_50D', 'SMA_200D', 'EMA_12D', 'EMA_26D', 'EMA_50D', 'EMA_200D', 
                'Volume', 'VMA_20D', 'VMA_50D', 'VMA_200D', 'BETA', 
                'VWMA_50D', 'VWMA_200D', 'MACD_12D_26D', 'RSI_14D', 'UPLOAD_STAT']]
    all_tech_dfs.append(df)
    return 


Load Current Profile

In [4]:
def load_sec_info_profile(ticker, stock):
    info = stock.info
    df = {'TICKER': ticker,
          'SEC_TYPE': info.get("quoteType", None),
          'COMPANY_NAME': info.get('shortName', None), 
          'SECTOR': info.get('sector', None),
          'INDUSTRY': info.get('industry', None)}
    df = pd.DataFrame([df])
    all_sec_info.append(df)
    return 


In [5]:
def load_current_technicals_stock_profile(pos_date, ticker, stock):
    
    info = stock.info
    history = stock.history(period="1y")
    delta = history['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    rs = gain.rolling(window=14).mean()[-1] / loss.rolling(window=14).mean()[-1]
    ema_12d = history['Close'].ewm(span=12, adjust=False).mean()[-1]
    ema_26d = history['Close'].ewm(span=26, adjust=False).mean()[-1]

    df = {'POS_DATE': pos_date.ISO(),
          'TICKER': ticker,
          'MARKET_CAP': info.get('marketCap'), 
          'OPEN': info.get('open'), 
          'HIGH': info.get('dayHigh'), 
          'LOW': info.get('dayLow'), 
          'CLOSE': info.get('previousClose'), 
          'SMA_50D': info.get('fiftyDayAverage'), 
          'SMA_200D': info.get('twoHundredDayAverage'), 
          'EMA_12D': ema_12d,
          'EMA_26D': ema_26d,
          'EMA_50D': history['Close'].ewm(span=50, adjust=False).mean()[-1],
          'EMA_200D': history['Close'].ewm(span=200, adjust=False).mean()[-1],
          'VOLUME': info.get('volume'), 
          'VMA_20D': history['Volume'].rolling(window=20).mean()[-1],
          'VMA_50D': history['Volume'].rolling(window=50).mean()[-1],
          'VMA_200D': history['Volume'].rolling(window=200).mean()[-1],
          'BETA': info.get('beta'),
          'VWMA_50D': (history['Close']*history['Volume'].rolling(window=50).sum() / history['Volume'].rolling(window=50).sum())[-1],
          'VWMA_200D': (history['Close']*history['Volume'].rolling(window=200).sum() / history['Volume'].rolling(window=200).sum())[-1],
          'MACD_12D_26D': ema_12d - ema_26d,
          'RSI_14D': 100 - (100 / (1 + rs)),
          'UPLOAD_STAT': 'AD' }
    df = pd.DataFrame([df])
    all_tech_dfs.append(df)
    return 

In [6]:

def load_current_fundamentals_stock_profile(pos_date, ticker, stock):
    
    info = stock.info
    balance_sheet = stock.balance_sheet
    financials = stock.financials 
    sector = info.get('sector', None)
    security_type = info.get("quoteType", None)
    df = {'POS_DATE': pos_date.ISO(),
          'TICKER': ticker,
          'FYE_DATE': financials.columns[0] if security_type == 'EQUITY' else None, 
          'EV': info.get('enterpriseValue'), 
          'EBIT': (financials.loc['Net Income'].iloc[0] if 'Financial' in sector else financials.loc['EBIT'].iloc[0]) if security_type == 'EQUITY' else None, 
          'NII': financials.loc['Net Interest Income'].iloc[0] if security_type == 'EQUITY' else None, 
          'EPS': info.get('trailingEps', None), 
          'PE_RATIO': info.get('trailingPE', None), 
          'PEG_RATIO': info.get('trailingPegRatio'),
          'ROE': info.get('returnOnEquity'), 
          'DC_RATIO': balance_sheet.loc['Total Debt'].iloc[0] / (balance_sheet.loc['Total Debt'].iloc[0] + balance_sheet.loc['Stockholders Equity'].iloc[0]) 
                                          if security_type == 'EQUITY' else None, 
          'ICR': ((financials.loc['Net Income'].iloc[0] if 'Financial' in sector else financials.loc['EBIT'].iloc[0]) 
                  / financials.loc['Interest Expense'].iloc[0]) if security_type == 'EQUITY' else None, 
          'EV_to_EBIT': info.get('enterpriseValue') / (financials.loc['Net Income'].iloc[0] if 'Financial' in sector else financials.loc['EBIT'].iloc[0]) 
                                          if security_type == 'EQUITY' else None, 
          'OPERATING_MARGIN': info.get('operatingMargins'), 
          'QUICK_RATIO': info.get('quickRatio'),
          'FWD_EPS': info.get('forwardEps', None),
          'FWD_PE': info.get('forwardPE', None), 
          'ANALYST_RECS': info.get('recommendationMean'),
          'UPLOAD_STAT': 'AD' }
   
    df = pd.DataFrame([df])
    all_fund_dfs.append(df)
    return 

In [7]:
def upload_fundamental_technical_snaps(pos_date, ticker, stock):
    print(f'Stock: {ticker}, {stock}')
    # Preliminary Date Check for Yahoo Finance
    history = stock.history(period="1d")
    if Date_Obj.ql_vs_timestamp(pos_date, history.index[-1]): 
        load_current_technicals_stock_profile(pos_date, ticker, stock)
        load_current_fundamentals_stock_profile(pos_date, ticker, stock)
    else:
        print(f'YahooFinance may not be updated for current date for ticker {ticker}. Ejecting...')
    return
    

In [8]:
def add_new_ticker_historical_technical_snaps(pos_date, ticker, stock, beta_df, start_date=db_start_date):
    load_sec_info_profile(ticker, stock)
    end_date = Date_Obj.calendar.advance(pos_date, -1, ql.Days)
    print(f"Loading historical technicals data for '{ticker}'...\nStart Date: {start_date}\nEnd Date: {end_date}")
    load_historical_technicals_stock_profile(ticker, stock, start_date, end_date, beta_df)
    load_historical_fundamentals_stock_profile(ticker, stock, start_date, end_date)
    return

def add_new_ticker_historical_fundamental_snaps(pos_date, ticker, stock, start_date=db_start_date):
    end_date = Date_Obj.calendar.advance(pos_date, -1, ql.Days)
    print(f"Loading historical fundamentals data for '{ticker}'...\nStart Date: {start_date}\nEnd Date: {end_date}")
    load_historical_fundamentals_stock_profile(ticker, stock, start_date, end_date)
    return


In [9]:
@jit
def simple_beta_fn(w, sample=False):
    if np.isnan(w).any(): return np.nan
    snp = w[:,0]
    ticker = w[:,1]
    n = len(snp)
    if sample: n-=1
    x_mean = np.sum(snp) / n
    y_mean = np.sum(ticker) / n
    cov = np.sum((snp - x_mean) * (ticker - y_mean)) / n
    var = np.sum((snp - x_mean) ** 2) / n
    if var == 0: return np.nan
    return cov/var

# https://investexcel.net/how-does-yahoo-finance-calculate-beta/
def get_historical_beta(pos_date, new_tickers):
    beta_tickers = ["^GSPC"]+list(new_tickers)
    beta_df_raw = yf.download(beta_tickers, start=db_beta_start_date.ISO(), end=pos_date.ISO(), interval="1d")['Close']
    beta_df = beta_df_raw.resample("MS").first()
    beta_df['FOM'] = beta_df.index.map(lambda x: Date_Obj.dt_to_ql(x))
    for col in beta_tickers:
        beta_df[f'LAST_{col}'] = beta_df[col].shift(1)
        beta_df[f'{col}_RTN'] = (beta_df[col]-beta_df[f'LAST_{col}'])/beta_df[f'LAST_{col}']
    for ticker in new_tickers:
        beta_df[f'{ticker}_BETA'] = beta_df[['^GSPC_RTN', f'{ticker}_RTN']].rolling(
                                    window=36, min_periods=36, method='table').apply(
                                    simple_beta_fn, raw=True, engine='numba')[f'{ticker}_RTN']
    return beta_df


In [ ]:

def process_portfolio_snaps(pos_date, tickers):
    start_time = dt.datetime.now()
    query = """select distinct ticker from SEC_INFO"""
    all_portfolio_tickers = list(DB_Obj.read_sql_qry(query)['TICKER'])

    threads = []
    all_tech_dfs, all_fund_dfs = [], []
    new_tickers = set(tickers)-set(all_portfolio_tickers)
    if len(new_tickers) != 0:
        print("Adding new tickers in DB...")
        all_new_tickers_stocks = yf.Tickers(new_tickers)
        beta_df = get_historical_beta(pos_date, new_tickers)
        for ticker in new_tickers:
            thread = Thread(target=add_new_ticker_historical_technical_snaps, 
                            args=(pos_date, ticker, all_new_tickers_stocks.tickers[ticker], beta_df))
            threads.append(thread)
            thread.start()
        for thread in threads:
            thread.join() 
        for info_df, tech_df in list(zip(all_sec_info, tech_df)):
            DB_Obj.upload_to_db(info_df, 'SEC_INFO')
            DB_Obj.upload_to_db(tech_df, 'TECHNICAL_SNAPS')

        for ticker in new_tickers:
            thread = Thread(target=add_new_ticker_historical_fundamental_snaps, 
                            args=(pos_date, ticker, all_new_tickers_stocks.tickers[ticker]))
            threads.append(thread)
            thread.start()
        for thread in threads:
            thread.join() 
        for fund_df in all_fund_dfs:
            DB_Obj.upload_to_db(fund_df, 'FUNDAMENTAL_SNAPS')

    print(f"""Adding {pos_date.ISO()} data in DB...""")
    threads = []
    all_tech_dfs, all_fund_dfs = [], []
    tickers= tickers[0:2]
    print(tickers)
    all_tickers_stocks = yf.Tickers(tickers)
    for ticker in tickers:
        thread = Thread(target=upload_fundamental_technical_snaps, 
                        args=(pos_date, ticker, all_tickers_stocks.tickers[ticker]))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join() 
        
    for tech_df, fund_df in list(zip(all_tech_dfs, all_fund_dfs)):
        print(tech_df, fund_df)
        DB_Obj.upload_to_db(tech_df, 'TECHNICAL_SNAPS')
        DB_Obj.upload_to_db(fund_df, 'FUNDAMENTAL_SNAPS')

    end_time = dt.datetime.now()
   
    print(f"Finished Uploading {pos_date.ISO()} in {end_time-start_time} secs!")
    return 
    

Main RUNME: Stock Ticker Inputs

In [ ]:
ticker_list = list(DB_Obj.read_sql_qry("""select distinct TICKER from SEC_INFO order by sec_type, sector, ticker""").loc[:, 'TICKER'])
process_portfolio_snaps(pos_date, ticker_list)

In [135]:
sec_info_df = DB_Obj.read_sql_qry("""select * from SEC_INFO""")
tech_df = DB_Obj.read_sql_qry(f"""select * from TECHNICAL_SNAPS order by ticker, pos_date""")
fund_df = DB_Obj.read_sql_qry(f"""select * from FUNDAMENTAL_SNAPS order by ticker, pos_date""")
with pd.ExcelWriter("Daily_Snaps.xlsx") as writer:  
    sec_info_df.to_excel(writer, sheet_name='SEC_INFO')
    fund_df.to_excel(writer, sheet_name='FUNDAMENTAL_SNAPS')
    tech_df.to_excel(writer, sheet_name='TECHNICAL_SNAPS')

In [ ]:
DB_Obj.close_conn()